# Liquidation Analysis - Dataframe-Based

This notebook performs comprehensive analysis of AAVE liquidation events and stores all results in pandas DataFrames for easy post-analysis. All dataframes are saved to CSV files for further analysis.

## Analysis Sections:
1. **Liquidation Events**: Individual liquidation events with PNL calculations
2. **Block Summary**: Aggregated statistics by block
3. **Liquidator Statistics**: Performance metrics by liquidator
4. **Flash Loan Analysis**: Detection of flash loans and price manipulation indicators
5. **Oracle Manipulation Analysis**: Detection of oracle price manipulation patterns


In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
import json
from datetime import datetime
from collections import defaultdict
import pandas as pd
import numpy as np
import os

# Add project root to Python path
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    
from utils import (
    get_infura_web3, 
    get_transactions_by_event_hash, 
    get_liquidation_from_tx_hash,
    calculate_liquidation_pnl_usd,
    get_token_decimals,
    get_flash_loans_in_block,
    FLASH_LOAN_EVENT_HASHES,
    detect_oracle_manipulation,
    retry_with_backoff,
    get_token_price_coingecko
)

# Create data directory if it doesn't exist
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("Imports loaded successfully")
print(f"Data directory: {DATA_DIR}")


Imports loaded successfully
Data directory: /Users/franklinliu/Projects/FinM31200-Final-Project/data


In [ ]:
# Connect to Ethereum via Infura
w3 = get_infura_web3(api_key=os.getenv("INFURA_API_KEY"))

print(f"Connected: {w3.is_connected()}")
print(f"Chain ID: {w3.eth.chain_id}")

# Get latest block and calculate block range
latest_block = w3.eth.block_number
from_block = latest_block - 100000
to_block = latest_block

print(f"\nLatest block: {latest_block}")
print(f"Searching blocks: {from_block} to {to_block} (inclusive)")
print(f"Total blocks to search: {to_block - from_block + 1}")

# Event hash to search
AAVE_LIQUIDATION_HASH = "0xe413a321e8681d831f4dbccbca790d2952b56f977908e45be37335533e005286"
print(f"\nAAVE Liquidation Event Hash: {AAVE_LIQUIDATION_HASH}")


Connected: True
Chain ID: 1

Latest block: 23969812
Searching blocks: 23869812 to 23969812 (inclusive)
Total blocks to search: 100001

AAVE Liquidation Event Hash: 0xe413a321e8681d831f4dbccbca790d2952b56f977908e45be37335533e005286


In [4]:
# Search for AAVE liquidation events
print("Searching for AAVE liquidation events...")
aave_events = get_transactions_by_event_hash(
    w3,
    AAVE_LIQUIDATION_HASH,
    from_block,
    to_block,
    contract_addresses=None  # Search all contracts
)

print(f"Found {len(aave_events)} AAVE liquidation event(s)")

# Extract liquidation information from transaction hashes
print("\nExtracting liquidation information from transaction hashes...")
structured_results = []

for event in aave_events:
    tx_hash = event.get("transactionHash")
    if tx_hash:
        result = get_liquidation_from_tx_hash(w3, tx_hash, AAVE_LIQUIDATION_HASH)
        if result:
            structured_results.append(result)

print(f"Successfully decoded {len(structured_results)} events")


Searching for AAVE liquidation events...
Found 951 AAVE liquidation event(s)

Extracting liquidation information from transaction hashes...
Successfully decoded 951 events


## 1. Liquidation Events DataFrame

Create a comprehensive DataFrame with all liquidation event details including PNL calculations.


In [5]:
# Build Liquidation Events DataFrame
liquidation_rows = []

for liq in structured_results:
    debt_asset = liq.get("debtAsset", "")
    collateral_asset = liq.get("collateralAsset", "")
    debt_to_cover = int(liq.get("debtToCover", 0))
    collateral_amount = int(liq.get("liquidatedCollateralAmount", 0))
    
    # Get correct decimals for each token
    debt_decimals = get_token_decimals(debt_asset) if debt_asset else 18
    collateral_decimals = get_token_decimals(collateral_asset) if collateral_asset else 18
    
    # Get timestamp for historical price lookup
    liquidation_timestamp = liq.get("timestamp")
    
    # Calculate PNL in USD
    pnl_data = calculate_liquidation_pnl_usd(
        debt_asset, debt_to_cover,
        collateral_asset, collateral_amount,
        debt_decimals=debt_decimals,
        collateral_decimals=collateral_decimals,
        timestamp=liquidation_timestamp
    )
    
    row = {
        "transactionHash": liq.get("transactionHash", ""),
        "blockNumber": liq.get("blockNumber"),
        "timestamp": liquidation_timestamp,
        "datetime": liq.get("datetime", ""),
        "liquidator": liq.get("liquidator", ""),
        "liquidatedUser": liq.get("liquidatedUser", ""),
        "debtAsset": debt_asset,
        "collateralAsset": collateral_asset,
        "debtToCover": debt_to_cover,
        "liquidatedCollateralAmount": collateral_amount,
        "debtDecimals": debt_decimals,
        "collateralDecimals": collateral_decimals,
        "debtUSD": pnl_data.get("debt_usd"),
        "collateralUSD": pnl_data.get("collateral_usd"),
        "pnlUSD": pnl_data.get("pnl_usd"),
    }
    liquidation_rows.append(row)

# Create DataFrame
df_liquidation_events = pd.DataFrame(liquidation_rows)

# Convert datetime column
if "datetime" in df_liquidation_events.columns:
    df_liquidation_events["datetime"] = pd.to_datetime(df_liquidation_events["datetime"], errors="coerce")

# Save to CSV
output_file = DATA_DIR / "liquidation_events.csv"
df_liquidation_events.to_csv(output_file, index=False)
print(f"✓ Liquidation Events DataFrame created: {len(df_liquidation_events)} rows")
print(f"✓ Saved to: {output_file}")
print(f"\nDataFrame shape: {df_liquidation_events.shape}")
print(f"\nColumns: {list(df_liquidation_events.columns)}")
print(f"\nFirst few rows:")
df_liquidation_events.head()


Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 4.8s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limited. Waiting 2.4s before retry...
Rate limite

,transactionHash,blockNumber,timestamp,datetime,liquidator,liquidatedUser,debtAsset,collateralAsset,debtToCover,liquidatedCollateralAmount,debtDecimals,collateralDecimals,debtUSD,collateralUSD,pnlUSD
0,0xba0531e21f2a6b3402164b784e80910aa92c1f6cefcd...,23866576,1763961311,2025-11-23 23:15:11,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,0x5EE4C8Af6d8bf62982E6721ff639eF6Dad41DF87,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0x6B175474E89094C44Da98b954EedeAC495271d0F,2665212898398913,7965391415744071644,18,18,7.377495,7.960892,0.583397
1,0x88eab56eecc6672bcac1bc4cf107641539fb916af691...,23866593,1763961515,2025-11-23 23:18:35,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,0xf11F56b30D839cE3A45c723212E9386C679dd4CA,0xdAC17F958D2ee523a2206206994597C13D831ec7,0x6B175474E89094C44Da98b954EedeAC495271d0F,39765468,41354218359225659996,6,18,39.749607,41.330856,1.581249
2,0x8cfc0595702ceeed2da5206baafdbffba0babd78df77...,23867052,1763967083,2025-11-24 00:51:23,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,0xDa632e0B993fC65EB62B84cFF3E9B1b9e51c63B1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,470205949580983545,474437803127212396,18,18,1316.828785,1328.680244,11.851459
3,0x846d144e05c050ef52387115ca2d7b7e09f77440b967...,23870214,1764005495,2025-11-24 11:31:35,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,0x21e7824340C276735a033b1bC45652EbBe007193,0x514910771AF9Ca656af840dff83E8264EcF986CA,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,2091643214136363636363,28131412986,18,6,26160.227023,28123.060233,1962.833210
4,0x125e2d9530710155bf8b5018568e06f370454dfe29a4...,23870833,1764013115,2025-11-24 13:38:35,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,0x2476F8D5828be822CfF24F38984e170fD9FA8b0e,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,10370743,10744089,6,6,10.367664,10.740899,0.373235


In [6]:
# Aggregate by block
block_summary_rows = []

for block_num in df_liquidation_events["blockNumber"].unique():
    block_data = df_liquidation_events[df_liquidation_events["blockNumber"] == block_num]
    
    # Get timestamp and datetime from first row (all same block)
    first_row = block_data.iloc[0]
    timestamp = first_row.get("timestamp")
    datetime_val = first_row.get("datetime")
    
    row = {
        "blockNumber": block_num,
        "timestamp": timestamp,
        "datetime": datetime_val,
        "totalLiquidations": len(block_data),
        "totalPNLUSD": block_data["pnlUSD"].sum() if block_data["pnlUSD"].notna().any() else None,
        "uniqueLiquidators": block_data["liquidator"].nunique(),
        "uniqueUsersLiquidated": block_data["liquidatedUser"].nunique(),
    }
    block_summary_rows.append(row)

# Create DataFrame
df_block_summary = pd.DataFrame(block_summary_rows)

# Sort by block number
df_block_summary = df_block_summary.sort_values("blockNumber")

# Convert datetime column
if "datetime" in df_block_summary.columns:
    df_block_summary["datetime"] = pd.to_datetime(df_block_summary["datetime"], errors="coerce")

# Save to CSV
output_file = DATA_DIR / "block_summary.csv"
df_block_summary.to_csv(output_file, index=False)
print(f"✓ Block Summary DataFrame created: {len(df_block_summary)} rows")
print(f"✓ Saved to: {output_file}")
print(f"\nDataFrame shape: {df_block_summary.shape}")
print(f"\nFirst few rows:")
df_block_summary.head()


✓ Block Summary DataFrame created: 605 rows
✓ Saved to: /Users/franklinliu/Projects/FinM31200-Final-Project/data/block_summary.csv

DataFrame shape: (605, 7)

First few rows:


,blockNumber,timestamp,datetime,totalLiquidations,totalPNLUSD,uniqueLiquidators,uniqueUsersLiquidated
0,23866576,1763961311,2025-11-23 23:15:11,1,0.583397,1,1
1,23866593,1763961515,2025-11-23 23:18:35,1,1.581249,1,1
2,23867052,1763967083,2025-11-24 00:51:23,1,11.851459,1,1
3,23870214,1764005495,2025-11-24 11:31:35,1,1962.833210,1,1
4,23870833,1764013115,2025-11-24 13:38:35,1,0.373235,1,1


## 3. Liquidator Statistics DataFrame

Aggregate performance metrics by liquidator address.


In [7]:
# Aggregate by liquidator
liquidator_stats_rows = []

for liquidator in df_liquidation_events["liquidator"].unique():
    liquidator_data = df_liquidation_events[df_liquidation_events["liquidator"] == liquidator]
    
    total_pnl = liquidator_data["pnlUSD"].sum() if liquidator_data["pnlUSD"].notna().any() else 0
    count = len(liquidator_data)
    avg_pnl = total_pnl / count if count > 0 else 0
    
    blocks = liquidator_data["blockNumber"].unique()
    
    row = {
        "liquidator": liquidator,
        "totalLiquidations": count,
        "totalPNLUSD": total_pnl,
        "averagePNLUSD": avg_pnl,
        "blocksActive": len(blocks),
        "firstLiquidationBlock": int(blocks.min()) if len(blocks) > 0 else None,
        "lastLiquidationBlock": int(blocks.max()) if len(blocks) > 0 else None,
    }
    liquidator_stats_rows.append(row)

# Create DataFrame
df_liquidator_stats = pd.DataFrame(liquidator_stats_rows)

# Sort by total PNL descending
df_liquidator_stats = df_liquidator_stats.sort_values("totalPNLUSD", ascending=False).reset_index(drop=True)

# Save to CSV
output_file = DATA_DIR / "liquidator_stats.csv"
df_liquidator_stats.to_csv(output_file, index=False)
print(f"✓ Liquidator Statistics DataFrame created: {len(df_liquidator_stats)} rows")
print(f"✓ Saved to: {output_file}")
print(f"\nDataFrame shape: {df_liquidator_stats.shape}")
print(f"\nTop liquidators:")
df_liquidator_stats.head()


✓ Liquidator Statistics DataFrame created: 43 rows
✓ Saved to: /Users/franklinliu/Projects/FinM31200-Final-Project/data/liquidator_stats.csv

DataFrame shape: (43, 7)

Top liquidators:


,liquidator,totalLiquidations,totalPNLUSD,averagePNLUSD,blocksActive,firstLiquidationBlock,lastLiquidationBlock
0,0xcb7066289252875360Ad69CE71514AEc41D50993,9,789287.689316,87698.632146,5,23919512,23961543
1,0x193e08Fd4375444017CAFF5f24C0FBC3f273f846,17,127755.107063,7515.006298,17,23877208,23961548
2,0xE08D97e151473A848C3d9CA3f323Cb720472D015,171,60662.429233,354.751048,87,23874053,23955916
3,0x88eb6e4358E940EeA0151E16C77C336370708729,6,46548.845789,7758.140965,5,23885188,23926554
4,0x1f2F10D1C40777AE1Da742455c65828FF36Df387,34,13667.615278,401.988685,31,23866576,23963702


## 4. Flash Loan Analysis DataFrame

Detect flash loans in blocks with liquidations and identify potential price manipulation.


In [8]:
# Get unique blocks with liquidations
blocks_with_liquidations = df_liquidation_events["blockNumber"].unique().tolist()

print(f"Searching for flash loans in {len(blocks_with_liquidations)} blocks with liquidations...")

# Collect flash loans by block
flash_loans_by_block = {}
for block_num in blocks_with_liquidations:
    flash_loans = get_flash_loans_in_block(w3, block_num)
    if flash_loans:
        flash_loans_by_block[block_num] = flash_loans

print(f"Found flash loans in {len(flash_loans_by_block)} block(s)")

# Build Flash Loan Analysis DataFrame
flash_loan_rows = []

for _, liq_row in df_liquidation_events.iterrows():
    tx_hash = liq_row["transactionHash"]
    block_num = liq_row["blockNumber"]
    liquidator = liq_row["liquidator"]
    debt_asset = liq_row["debtAsset"]
    collateral_asset = liq_row["collateralAsset"]
    
    # Get flash loans in this block
    block_flash_loans = flash_loans_by_block.get(block_num, [])
    
    if block_flash_loans:
        # Check each flash loan for matches
        for fl in block_flash_loans:
            fl_tx_hash = fl.get("transactionHash", "")
            fl_initiator = fl.get("initiator", "").lower()
            fl_asset = fl.get("asset", "").lower()
            
            # Check for token match (debt or collateral)
            token_match = (
                fl_asset == debt_asset.lower() or 
                fl_asset == collateral_asset.lower()
            )
            
            # Check for address match (liquidator = flash loan initiator)
            address_match = fl_initiator == liquidator.lower()
            
            # Determine if manipulation is detected
            manipulation_detected = token_match and address_match
            
            row = {
                "liquidationTxHash": tx_hash,
                "liquidationBlock": block_num,
                "flashLoanTxHash": fl_tx_hash,
                "flashLoanProtocol": fl.get("protocol", ""),
                "flashLoanInitiator": fl.get("initiator", ""),
                "flashLoanAsset": fl.get("asset", ""),
                "flashLoanAmount": fl.get("amount", 0),
                "flashLoanPremium": fl.get("premium", 0),
                "tokenMatch": token_match,
                "addressMatch": address_match,
                "manipulationDetected": manipulation_detected,
            }
            flash_loan_rows.append(row)
    else:
        # No flash loans in this block
        row = {
            "liquidationTxHash": tx_hash,
            "liquidationBlock": block_num,
            "flashLoanTxHash": None,
            "flashLoanProtocol": None,
            "flashLoanInitiator": None,
            "flashLoanAsset": None,
            "flashLoanAmount": None,
            "flashLoanPremium": None,
            "tokenMatch": False,
            "addressMatch": False,
            "manipulationDetected": False,
        }
        flash_loan_rows.append(row)

# Create DataFrame
df_flash_loan_analysis = pd.DataFrame(flash_loan_rows)

# Save to CSV
output_file = DATA_DIR / "flash_loan_analysis.csv"
df_flash_loan_analysis.to_csv(output_file, index=False)
print(f"✓ Flash Loan Analysis DataFrame created: {len(df_flash_loan_analysis)} rows")
print(f"✓ Saved to: {output_file}")
print(f"\nDataFrame shape: {df_flash_loan_analysis.shape}")
print(f"\nManipulation detected cases: {df_flash_loan_analysis['manipulationDetected'].sum()}")
print(f"\nFirst few rows:")
df_flash_loan_analysis.head()


Searching for flash loans in 605 blocks with liquidations...
Found flash loans in 296 block(s)
✓ Flash Loan Analysis DataFrame created: 2630 rows
✓ Saved to: /Users/franklinliu/Projects/FinM31200-Final-Project/data/flash_loan_analysis.csv

DataFrame shape: (2630, 11)

Manipulation detected cases: 794

First few rows:


,liquidationTxHash,liquidationBlock,flashLoanTxHash,flashLoanProtocol,flashLoanInitiator,flashLoanAsset,flashLoanAmount,flashLoanPremium,tokenMatch,addressMatch,manipulationDetected
0,0xba0531e21f2a6b3402164b784e80910aa92c1f6cefcd...,23866576,None,None,None,None,NaN,NaN,False,False,False
1,0x88eab56eecc6672bcac1bc4cf107641539fb916af691...,23866593,dc5cc9b2c6a4d0b40efce3bce0da17c4138e8105144f2b...,Balancer,0xAFb55F4155E0729dd7Da101D6Ef958B3645FD070,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,1.000000e+17,0.000000e+00,False,False,False
2,0x8cfc0595702ceeed2da5206baafdbffba0babd78df77...,23867052,915a9e4b91d42cb0b31c797a83c4fe94d49c608c07e230...,Balancer,0x0000000000EfA780a8e6f50fC5De9c1497BFd175,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2.242068e+17,0.000000e+00,True,False,False
3,0x846d144e05c050ef52387115ca2d7b7e09f77440b967...,23870214,0be1164711b4e69709e7a7ffef54d2a9f7413c41b25522...,Aave V3,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0x0000000000000000000000000000000000000000,6.459189e+47,4.000000e+18,False,False,False
4,0x125e2d9530710155bf8b5018568e06f370454dfe29a4...,23870833,None,None,None,None,NaN,NaN,False,False,False


## 5. Oracle Manipulation Analysis DataFrame

Detect oracle price manipulation by analyzing DEX swaps, oracle updates, and flash loans in liquidation blocks.


In [9]:
# Build Oracle Manipulation Analysis DataFrame
oracle_manipulation_rows = []

print(f"Analyzing {len(df_liquidation_events)} liquidation(s) for oracle manipulation...")

for idx, liq_row in df_liquidation_events.iterrows():
    tx_hash = liq_row["transactionHash"]
    block_num = liq_row["blockNumber"]
    
    # Get liquidation data dict
    liq_data = {
        "liquidator": liq_row["liquidator"],
        "liquidatedUser": liq_row["liquidatedUser"],
        "debtAsset": liq_row["debtAsset"],
        "collateralAsset": liq_row["collateralAsset"],
        "debtToCover": liq_row["debtToCover"],
        "liquidatedCollateralAmount": liq_row["liquidatedCollateralAmount"],
    }
    
    # Get flash loans in this block
    block_flash_loans = flash_loans_by_block.get(block_num, [])
    
    # Detect manipulation indicators (scan entire block)
    indicators = detect_oracle_manipulation(
        w3,
        tx_hash,
        liq_data,
        block_number=block_num,
        flash_loans=block_flash_loans if block_flash_loans else None,
        scan_block=True  # Scan entire block, not just transaction
    )
    
    row = {
        "liquidationTxHash": tx_hash,
        "liquidationBlock": block_num,
        "hasFlashLoan": indicators.get("has_flash_loan", False),
        "hasLargeSwaps": indicators.get("has_large_swaps", False),
        "hasOracleUpdates": indicators.get("has_oracle_updates", False),
        "swapCount": len(indicators.get("swap_details", [])),
        "oracleUpdateCount": len(indicators.get("oracle_update_details", [])),
        "flashLoanCount": len(indicators.get("flash_loan_details", [])),
        "manipulationScore": indicators.get("manipulation_score", 0),
        "likelyManipulation": indicators.get("likely_manipulation", False),
        "flags": json.dumps(indicators.get("flags", [])),  # Store as JSON string
        "scanScope": indicators.get("scan_scope", "block"),
    }
    oracle_manipulation_rows.append(row)
    print(row)
    if (idx + 1) % 5 == 0:
        print(f"  Processed {idx + 1}/{len(df_liquidation_events)} liquidations...")

print(f"✓ Completed analysis for {len(oracle_manipulation_rows)} liquidation(s)")

# Create DataFrame
df_oracle_manipulation = pd.DataFrame(oracle_manipulation_rows)

# Save to CSV
output_file = DATA_DIR / "oracle_manipulation_analysis.csv"
df_oracle_manipulation.to_csv(output_file, index=False)
print(f"✓ Oracle Manipulation Analysis DataFrame created: {len(df_oracle_manipulation)} rows")
print(f"✓ Saved to: {output_file}")
print(f"\nDataFrame shape: {df_oracle_manipulation.shape}")
print(f"\nLikely manipulation cases: {df_oracle_manipulation['likelyManipulation'].sum()}")
print(f"\nAverage manipulation score: {df_oracle_manipulation['manipulationScore'].mean():.2f}")
print(f"\nFirst few rows:")
df_oracle_manipulation.head()


Analyzing 951 liquidation(s) for oracle manipulation...
{'liquidationTxHash': '0xba0531e21f2a6b3402164b784e80910aa92c1f6cefcd6076fa3ce9524bd5bea7', 'liquidationBlock': 23866576, 'hasFlashLoan': False, 'hasLargeSwaps': True, 'hasOracleUpdates': False, 'swapCount': 83, 'oracleUpdateCount': 0, 'flashLoanCount': 0, 'manipulationScore': 3, 'likelyManipulation': False, 'flags': '["1 DEX swap(s) in same transaction", "82 DEX swap(s) in same block, different transaction(s)"]', 'scanScope': 'block'}
{'liquidationTxHash': '0x88eab56eecc6672bcac1bc4cf107641539fb916af6917f6f70eff5de1b71a7c9', 'liquidationBlock': 23866593, 'hasFlashLoan': True, 'hasLargeSwaps': True, 'hasOracleUpdates': True, 'swapCount': 29, 'oracleUpdateCount': 1, 'flashLoanCount': 1, 'manipulationScore': 8, 'likelyManipulation': True, 'flags': '["Flash loan in same block, different transaction (1 loan(s))", "6 DEX swap(s) in same transaction", "23 DEX swap(s) in same block, different transaction(s)", "1 oracle update(s) in same 

,liquidationTxHash,liquidationBlock,hasFlashLoan,hasLargeSwaps,hasOracleUpdates,swapCount,oracleUpdateCount,flashLoanCount,manipulationScore,likelyManipulation,flags,scanScope
0,0xba0531e21f2a6b3402164b784e80910aa92c1f6cefcd...,23866576,False,True,False,83,0,0,3,False,"[""1 DEX swap(s) in same transaction"", ""82 DEX ...",block
1,0x88eab56eecc6672bcac1bc4cf107641539fb916af691...,23866593,True,True,True,29,1,1,8,True,"[""Flash loan in same block, different transact...",block
2,0x8cfc0595702ceeed2da5206baafdbffba0babd78df77...,23867052,True,True,False,55,0,1,5,True,"[""Flash loan in same block, different transact...",block
3,0x846d144e05c050ef52387115ca2d7b7e09f77440b967...,23870214,True,True,True,23,1,1,8,True,"[""Flash loan in same block, different transact...",block
4,0x125e2d9530710155bf8b5018568e06f370454dfe29a4...,23870833,False,True,False,54,0,0,1,False,"[""54 DEX swap(s) in same block, different tran...",block


## Summary Statistics

Display overview statistics for all dataframes.


In [10]:
print("=" * 100)
print("DATAFRAME SUMMARY STATISTICS")
print("=" * 100)

print("\n1. LIQUIDATION EVENTS")
print("-" * 100)
print(f"  Total liquidations: {len(df_liquidation_events)}")
print(f"  Date range: {df_liquidation_events['datetime'].min()} to {df_liquidation_events['datetime'].max()}")
print(f"  Unique liquidators: {df_liquidation_events['liquidator'].nunique()}")
print(f"  Unique users liquidated: {df_liquidation_events['liquidatedUser'].nunique()}")
print(f"  Total PNL (USD): ${df_liquidation_events['pnlUSD'].sum():,.2f}")
print(f"  Average PNL (USD): ${df_liquidation_events['pnlUSD'].mean():,.2f}")

print("\n2. BLOCK SUMMARY")
print("-" * 100)
print(f"  Total blocks with liquidations: {len(df_block_summary)}")
print(f"  Total liquidations across blocks: {df_block_summary['totalLiquidations'].sum()}")
print(f"  Average liquidations per block: {df_block_summary['totalLiquidations'].mean():.2f}")
print(f"  Total PNL across blocks: ${df_block_summary['totalPNLUSD'].sum():,.2f}")

print("\n3. LIQUIDATOR STATISTICS")
print("-" * 100)
print(f"  Total unique liquidators: {len(df_liquidator_stats)}")
print(f"  Top liquidator: {df_liquidator_stats.iloc[0]['liquidator']}")
print(f"  Highest total PNL: ${df_liquidator_stats.iloc[0]['totalPNLUSD']:,.2f}")
print(f"  Average liquidations per liquidator: {df_liquidator_stats['totalLiquidations'].mean():.2f}")

print("\n4. FLASH LOAN ANALYSIS")
print("-" * 100)
print(f"  Total records: {len(df_flash_loan_analysis)}")
print(f"  Liquidations with flash loans: {(df_flash_loan_analysis['flashLoanTxHash'].notna()).sum()}")
print(f"  Token matches detected: {df_flash_loan_analysis['tokenMatch'].sum()}")
print(f"  Address matches detected: {df_flash_loan_analysis['addressMatch'].sum()}")
print(f"  Manipulation detected: {df_flash_loan_analysis['manipulationDetected'].sum()}")

print("\n5. ORACLE MANIPULATION ANALYSIS")
print("-" * 100)
print(f"  Total records: {len(df_oracle_manipulation)}")
print(f"  Cases with flash loans: {df_oracle_manipulation['hasFlashLoan'].sum()}")
print(f"  Cases with swaps: {df_oracle_manipulation['hasLargeSwaps'].sum()}")
print(f"  Cases with oracle updates: {df_oracle_manipulation['hasOracleUpdates'].sum()}")
print(f"  Likely manipulation cases: {df_oracle_manipulation['likelyManipulation'].sum()}")
print(f"  Average manipulation score: {df_oracle_manipulation['manipulationScore'].mean():.2f}/10")

print("\n" + "=" * 100)
print("All dataframes saved to CSV files in the data/ directory")
print("=" * 100)


DATAFRAME SUMMARY STATISTICS

1. LIQUIDATION EVENTS
----------------------------------------------------------------------------------------------------
  Total liquidations: 951
  Date range: 2025-11-23 23:15:11 to 2025-12-07 18:09:11
  Unique liquidators: 43
  Unique users liquidated: 599
  Total PNL (USD): $1,057,126.64
  Average PNL (USD): $1,118.65

2. BLOCK SUMMARY
----------------------------------------------------------------------------------------------------
  Total blocks with liquidations: 605
  Total liquidations across blocks: 951
  Average liquidations per block: 1.57
  Total PNL across blocks: $1,057,126.64

3. LIQUIDATOR STATISTICS
----------------------------------------------------------------------------------------------------
  Total unique liquidators: 43
  Top liquidator: 0xcb7066289252875360Ad69CE71514AEc41D50993
  Highest total PNL: $789,287.69
  Average liquidations per liquidator: 22.12

4. FLASH LOAN ANALYSIS
----------------------------------------------

In [11]:
# Show manipulation cases together with pnl

# Merge likely manipulation cases with liquidation events (to get PNL info)
# First, ensure both DataFrames have correct keys and are not empty
if not df_oracle_manipulation.empty and not df_liquidation_events.empty:
    # We assume df_oracle_manipulation has 'liquidationTxHash' and df_liquidation_events has 'transactionHash'
    df_manip_likely = df_oracle_manipulation[df_oracle_manipulation['likelyManipulation']]
    # Merge on the tx hash columns
    df_manip_with_pnl = df_manip_likely.merge(
        df_liquidation_events,
        left_on='liquidationTxHash',
        right_on='transactionHash',
        how='left',
        suffixes=('', '_liq')
    )
    # Select relevant columns to display
    cols_to_show = [
        'liquidationTxHash', 'liquidationBlock',
        'hasFlashLoan', 'hasLargeSwaps', 'hasOracleUpdates',
        'manipulationScore', 'likelyManipulation', 'flags',
        'liquidator', 'liquidatedUser', 'pnlUSD'
    ]
    intersection_cols = [col for col in cols_to_show if col in df_manip_with_pnl.columns]
    display_cols = intersection_cols if intersection_cols else df_manip_with_pnl.columns

    print("\nLIKELY MANIPULATION CASES WITH PNL")
    print("-" * 100)
    print(df_manip_with_pnl[display_cols].sort_values('manipulationScore', ascending=False).to_string(index=False))
else:
    print("\nNo manipulation or liquidation events found for joined analysis.")





LIKELY MANIPULATION CASES WITH PNL
----------------------------------------------------------------------------------------------------
                                                 liquidationTxHash  liquidationBlock  hasFlashLoan  hasLargeSwaps  hasOracleUpdates  manipulationScore  likelyManipulation                                                                                                                                                                                                                          flags                                 liquidator                             liquidatedUser        pnlUSD
0x88eab56eecc6672bcac1bc4cf107641539fb916af6917f6f70eff5de1b71a7c9          23866593          True           True              True                  8                True   ["Flash loan in same block, different transaction (1 loan(s))", "6 DEX swap(s) in same transaction", "23 DEX swap(s) in same block, different transaction(s)", "1 oracle update(s) in same block, di

## Enhanced Detailed Price Manipulation Analysis

This section provides detailed analysis of price manipulation with:
- **Price change tracking**: Oracle price updates with before/after values
- **Token-specific swaps**: Swaps filtered by debt/collateral tokens
- **PNL ranking**: Liquidations ranked by profit to identify most profitable manipulation cases


In [21]:
# Import the new detailed manipulation detection function
from utils import detect_detailed_price_manipulation

# Perform detailed manipulation analysis for all liquidations
print("Performing detailed price manipulation analysis...")
print("This may take a while as it queries pair contracts for token addresses...")
print("=" * 100)

detailed_manipulation_results = []

# Process liquidations - only check blocks where PNL is greater than $100k
pnl_threshold = 1000  # $100k USD
print(f"Filtering liquidations with PNL > ${pnl_threshold:,}...")

filtered_liquidations = df_liquidation_events[df_liquidation_events["pnlUSD"] > pnl_threshold]
print(f"Found {len(filtered_liquidations)} liquidations with PNL > ${pnl_threshold:,} out of {len(df_liquidation_events)} total")

for idx, row in filtered_liquidations.iterrows():
    
    tx_hash = row["transactionHash"]
    block_num = row["blockNumber"]
    
    # Get liquidation data
    liq_data = {
        "liquidator": row["liquidator"],
        "liquidatedUser": row["liquidatedUser"],
        "debtAsset": row["debtAsset"],
        "collateralAsset": row["collateralAsset"],
        "debtToCover": row["debtToCover"],
        "liquidatedCollateralAmount": row["liquidatedCollateralAmount"],
    }
    
    # Get flash loans for this block
    block_flash_loans = flash_loans_by_block.get(block_num, [])
    
    try:
        # Perform detailed analysis
        detailed_result = detect_detailed_price_manipulation(
            w3, 
            tx_hash,
            liq_data,
            block_num,
            flash_loans=block_flash_loans if block_flash_loans else None
        )
        
        # Add PNL information
        detailed_result["pnlUSD"] = row["pnlUSD"]
        detailed_result["debtUSD"] = row["debtUSD"]
        detailed_result["collateralUSD"] = row["collateralUSD"]
        
        detailed_manipulation_results.append(detailed_result)
        
        processed_count = len(detailed_manipulation_results)
        if processed_count % 5 == 0:
            print(f"Processed {processed_count}/{len(filtered_liquidations)} liquidations with PNL > ${pnl_threshold:,}...")
            
    except Exception as e:
        print(f"Warning: Error analyzing {tx_hash}: {e}")
        continue

print(f"\nCompleted detailed analysis for {len(detailed_manipulation_results)} liquidations")


Performing detailed price manipulation analysis...
This may take a while as it queries pair contracts for token addresses...
Filtering liquidations with PNL > $1,000...
Found 18 liquidations with PNL > $1,000 out of 951 total
Processed 5/18 liquidations with PNL > $1,000...
Processed 10/18 liquidations with PNL > $1,000...
Processed 15/18 liquidations with PNL > $1,000...

Completed detailed analysis for 18 liquidations


In [29]:
# Create DataFrame for detailed manipulation analysis
df_detailed_manipulation = pd.DataFrame([
    {
        "liquidationTxHash": r["liquidationTxHash"],
        "blockNumber": r["blockNumber"],
        "liquidator": r["liquidator"],
        "debtAsset": r["debtAsset"],
        "collateralAsset": r["collateralAsset"],
        "pnlUSD": r["pnlUSD"],
        "debtUSD": r.get("debtUSD"),
        "collateralUSD": r.get("collateralUSD"),
        "flashLoanCount": len(r["flashLoans"]),
        "oracleUpdateCount": len(r["oraclePriceUpdates"]),
        "oracleFeeds": json.dumps([update.get("feed", "Unknown") for update in r["oraclePriceUpdates"]]) if r["oraclePriceUpdates"] else None,
        "oraclePrices": json.dumps([update.get("priceUSD") for update in r["oraclePriceUpdates"] if update.get("priceUSD") is not None]) if r["oraclePriceUpdates"] else None,
        "oracleUpdatesInSameTx": len([update for update in r["oraclePriceUpdates"] if update.get("transactionHash", "").lower() == r["liquidationTxHash"].lower()]) if r["oraclePriceUpdates"] else 0,
        "oracleUpdatesDetails": json.dumps(r["oraclePriceUpdates"]) if r["oraclePriceUpdates"] else None,
        "debtTokenSwapCount": len(r["tokenSwaps"]["debtTokenSwaps"]),
        "collateralTokenSwapCount": len(r["tokenSwaps"]["collateralTokenSwaps"]),
        "totalSwapCount": len(r["tokenSwaps"]["allSwaps"]),
        "manipulationScore": r["manipulationScore"],
        "likelyManipulation": r["likelyManipulation"],
        "flags": json.dumps(r["flags"]) if r["flags"] else None,
    }
    for r in detailed_manipulation_results
])

# Sort by PNL descending
df_detailed_manipulation = df_detailed_manipulation.sort_values("pnlUSD", ascending=False, na_position="last")

# Save to CSV
df_detailed_manipulation.to_csv(DATA_DIR / "detailed_manipulation_analysis.csv", index=False)

print("Detailed manipulation analysis DataFrame created and saved!")
print(f"Shape: {df_detailed_manipulation.shape}")
print("\nTop 10 by PNL:")
print(df_detailed_manipulation.head(10)[["liquidationTxHash", "pnlUSD", "manipulationScore", "likelyManipulation", "flashLoanCount", "oracleUpdateCount", "oracleUpdatesInSameTx", "debtTokenSwapCount", "collateralTokenSwapCount"]].to_string(index=False))


Detailed manipulation analysis DataFrame created and saved!
Shape: (18, 20)

Top 10 by PNL:
                                                 liquidationTxHash        pnlUSD  manipulationScore  likelyManipulation  flashLoanCount  oracleUpdateCount  oracleUpdatesInSameTx  debtTokenSwapCount  collateralTokenSwapCount
0x74cd74d93e262e35061d01beb52fa32de7734c1a95428137be905802be02ea7f 376257.363965                192                True               2                  0                      0                  73                        17
0x74cd74d93e262e35061d01beb52fa32de7734c1a95428137be905802be02ea7f 376257.363965                192                True               2                  0                      0                  73                        17
0xd7626f820fb7b2e6a7af1cc54dd127fc8bbfa8ef5465cade8470dfe101e22166 125729.358713                 78                True               1                  0                      0                  33                         1
0xb51066cee3

### Top Manipulated Cases Ranked by PNL

Show the most profitable liquidation cases with detailed manipulation indicators.


### Detailed Analysis: Price Changes and Token-Specific Swaps

For the top manipulated cases, show detailed breakdown of oracle price updates and token-specific swaps.


In [30]:
# Filter for likely manipulation cases and rank by PNL
df_manipulated_ranked = df_detailed_manipulation[
    df_detailed_manipulation["likelyManipulation"] == True
].copy()

if not df_manipulated_ranked.empty:
    print("=" * 100)
    print("TOP MANIPULATED LIQUIDATIONS (Ranked by PNL)")
    print("=" * 100)
    
    for idx, row in df_manipulated_ranked.head(10).iterrows():
        rank = df_manipulated_ranked.index.get_loc(idx) + 1
        print(f"\n{'─' * 100}")
        print(f"Rank #{rank}: PNL ${row['pnlUSD']:,.2f}")
        print(f"  Transaction: {row['liquidationTxHash']}")
        print(f"  Block: {row['blockNumber']}")
        print(f"  Liquidator: {row['liquidator']}")
        print(f"  Debt Asset: {row['debtAsset']}")
        print(f"  Collateral Asset: {row['collateralAsset']}")
        print(f"  Manipulation Score: {row['manipulationScore']}/10")
        print(f"  Flash Loans: {row['flashLoanCount']}")
        print(f"  Oracle Updates: {row['oracleUpdateCount']}")
        print(f"  Debt Token Swaps: {row['debtTokenSwapCount']}")
        print(f"  Collateral Token Swaps: {row['collateralTokenSwapCount']}")
        print(f"  Total Swaps: {row['totalSwapCount']}")
        
        # Parse and display flags
        if row['flags']:
            flags = json.loads(row['flags'])
            print(f"  Flags ({len(flags)}):")
            for flag in flags[:5]:  # Show first 5 flags
                print(f"    - {flag}")
            if len(flags) > 5:
                print(f"    ... and {len(flags) - 5} more")
else:
    print("No likely manipulation cases found.")


TOP MANIPULATED LIQUIDATIONS (Ranked by PNL)

────────────────────────────────────────────────────────────────────────────────────────────────────
Rank #1: PNL $376,257.36
  Transaction: 0x74cd74d93e262e35061d01beb52fa32de7734c1a95428137be905802be02ea7f
  Block: 23926759
  Liquidator: 0xcb7066289252875360ad69ce71514aec41d50993
  Debt Asset: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
  Collateral Asset: 0xdac17f958d2ee523a2206206994597c13d831ec7
  Manipulation Score: 192/10
  Flash Loans: 2
  Oracle Updates: 0
  Debt Token Swaps: 73
  Collateral Token Swaps: 17
  Total Swaps: 105
  Flags (95):
    - Flash loan for liquidation token (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2)
    - Flash loan initiator matches liquidator
    - Swap involving collateral token (0xdac17f958d2ee523a2206206994597c13d831ec7)
    - Swap involving debt token (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2)
    - Swap involving debt token (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2)
    ... and 90 more

────────────

In [48]:
# Show detailed breakdown for top 3 manipulated cases
print("=" * 100)
print("DETAILED BREAKDOWN: Top 3 Manipulated Cases")
print("=" * 100)

top_3 = df_manipulated_ranked.head(3)

for rank, (idx, row) in enumerate(top_3.iterrows(), 1):
    tx_hash = row["liquidationTxHash"]
    
    # Find the detailed result
    detailed_result = next((r for r in detailed_manipulation_results if r["liquidationTxHash"] == tx_hash), None)
    
    if not detailed_result:
        continue
    
    print(f"\n{'=' * 100}")
    print(f"Case #{rank}: {tx_hash}")
    print(f"PNL: ${row['pnlUSD']:,.2f} | Score: {row['manipulationScore']}/10")
    print(f"{'=' * 100}")
    
    # Flash Loans
    print(f"\n📊 Flash Loans ({len(detailed_result['flashLoans'])}):")
    if detailed_result["flashLoans"]:
        for fl in detailed_result["flashLoans"][:3]:  # Show first 3
            asset = fl.get("asset", "Unknown")
            amount = fl.get("amount", 0)
            initiator = fl.get("initiator", "Unknown")
            print(f"  - Asset: {asset}")
            print(f"    Amount: {amount:,}")
            print(f"    Initiator: {initiator}")
            print(f"    Matches debt/collateral: {asset.lower() == row['debtAsset'].lower() or asset.lower() == row['collateralAsset'].lower()}")
            print(f"    Matches liquidator: {initiator.lower() == row['liquidator'].lower()}")
    else:
        print("  None")
    
    # Oracle Price Updates
    print(f"\n📈 Oracle Price Updates ({len(detailed_result['oraclePriceUpdates'])}):")
    if detailed_result["oraclePriceUpdates"]:
        for oracle in detailed_result["oraclePriceUpdates"][:3]:  # Show first 3
            feed = oracle.get("feed", "Unknown")
            price_usd = oracle.get("priceUSD")
            round_id = oracle.get("roundId")
            print(f"  - Feed: {feed}")
            print(f"    Price: ${price_usd:,.2f}" if price_usd else "    Price: N/A")
            print(f"    Round ID: {round_id}")
            print(f"    Oracle: {oracle.get('oracle', 'Unknown')}")
    else:
        print("  None")
    
    # Collateral Token Swaps
    print(f"\n💱 Collateral Token Swaps ({len(detailed_result['tokenSwaps']['collateralTokenSwaps'])}):")
    if detailed_result["tokenSwaps"]["collateralTokenSwaps"]:
        for swap in detailed_result["tokenSwaps"]["collateralTokenSwaps"][:3]:  # Show first 3
            protocol = swap.get("protocol", "Unknown")
            token0 = swap.get("token0", "Unknown")
            token1 = swap.get("token1", "Unknown")
            tx_swap = swap.get("transactionHash", "Unknown")
            print(f"  - Protocol: {protocol}")
            print(f"    Pair: {token0} / {token1}")
            print(f"    Transaction: {tx_swap[:20]}...")
            if "amount0In" in swap:
                print(f"    Amount0In: {swap.get('amount0In', 0):,}")
                print(f"    Amount1Out: {swap.get('amount1Out', 0):,}")
    else:
        print("  None")
    
    print()


DETAILED BREAKDOWN: Top 3 Manipulated Cases

Case #1: 0x74cd74d93e262e35061d01beb52fa32de7734c1a95428137be905802be02ea7f
PNL: $376,257.36 | Score: 192/10

📊 Flash Loans (2):
  - Asset: 0x0000000000000000000000000000000000000000
    Amount: 1,161,431,709,198,430,370,052,028,461,547,520,635,396,418,767,251
    Initiator: 0x514910771AF9Ca656af840dff83E8264EcF986CA
    Matches debt/collateral: False
    Matches liquidator: False
  - Asset: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
    Amount: 1,136,037,612,019,866,527,204
    Initiator: 0xcb7066289252875360Ad69CE71514AEc41D50993
    Matches debt/collateral: True
    Matches liquidator: True

📈 Oracle Price Updates (0):
  None

💱 Collateral Token Swaps (17):
  - Protocol: Uniswap V2
    Pair: 0x5B7533812759B45C2B44C19e320ba2cD2681b542 / 0xdAC17F958D2ee523a2206206994597C13D831ec7
    Transaction: 0xa8855b8d4984e824de...
    Amount0In: 0
    Amount1Out: 0
  - Protocol: Uniswap V2
    Pair: 0x88800092fF476844f74dC2FC427974BBee2794Ae / 0xdAC17

## Transaction Cost Analysis and Summary Statistics

Calculate net PnL after transaction costs and generate summary statistics.


In [59]:
w3.eth.get_transaction_receipt('0x74cd74d93e262e35061d01beb52fa32de7734c1a95428137be905802be02ea7f')

AttributeDict({'blockHash': HexBytes('0xc3251ba041b585e5505cc808e325f6397885d6d6f131a2812407f5d830eb5a8e'),
 'blockNumber': 23926759,
 'contractAddress': None,
 'cumulativeGasUsed': 2090461,
 'effectiveGasPrice': 925985354,
 'from': '0xE1dA407Dde7950654142e459a090B0eC5Ed2806B',
 'gasUsed': 1903301,
 'logs': [AttributeDict({'address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blockHash': HexBytes('0xc3251ba041b585e5505cc808e325f6397885d6d6f131a2812407f5d830eb5a8e'),
   'blockNumber': 23926759,
   'blockTimestamp': '0x692f09ab',
   'data': HexBytes('0x00000000000000000000000000000000000000000000003d95b05d25972de5e4'),
   'logIndex': 4,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000ba12222222228d8ba445958a75a0704d566bf2c8'),
    HexBytes('0x000000000000000000000000cb7066289252875360ad69ce71514aec41d50993')],
   'transactionHash': HexBytes('0x74cd74d93e262e35061d01beb52fa32d

In [63]:
# Load the liquidation events DataFrame
df_analysis = pd.read_csv(DATA_DIR / "liquidation_events.csv")

print(f"Loaded {len(df_analysis)} liquidation records")
print(f"Columns: {list(df_analysis.columns)}")

# Get transaction costs (gas fees) for each liquidation
print("\nFetching transaction costs...")
transaction_costs_usd = []
weth_address = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"

for idx, row in df_analysis.iterrows():
    tx_hash = row["transactionHash"]
    block_number = row["blockNumber"]
    tx_timestamp = row.get("timestamp")  # Use timestamp from CSV if available
    
    try:
        # Get transaction receipt
        receipt = retry_with_backoff(lambda: w3.eth.get_transaction_receipt(tx_hash))
        
        # Get gas used and effective gas price
        gas_used = receipt.get("gasUsed", 0)
        effective_gas_price = receipt.get("effectiveGasPrice", 0)
        
        # Use timestamp from CSV, or fetch from block if not available
        if pd.notna(tx_timestamp) and tx_timestamp > 0:
            block_timestamp = int(tx_timestamp)
        else:
            # Fallback: Get block timestamp for historical ETH price
            block = retry_with_backoff(lambda: w3.eth.get_block(block_number))
            block_timestamp = block.get("timestamp")
        
        # Get ETH price at time of transaction
        eth_price = get_token_price_coingecko(weth_address, timestamp=block_timestamp)
        
        if eth_price and gas_used and effective_gas_price:
            # Calculate transaction cost in ETH, then convert to USD
            # effective_gas_price is in wei, gas_used is in gas units
            # Cost in ETH = (gas_used * effective_gas_price) / 1e18
            cost_eth = (gas_used * effective_gas_price) / 1e18
            cost_usd = cost_eth * eth_price
        else:
            cost_usd = None
            
        transaction_costs_usd.append(cost_usd)
        
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1}/{len(df_analysis)} transactions...")
            
    except Exception as e:
        print(f"Error fetching transaction cost for {tx_hash}: {e}")
        transaction_costs_usd.append(None)

# Add transaction costs to DataFrame
df_analysis["transactionCostUSD"] = transaction_costs_usd

# Calculate net PnL (PnL - transaction costs)
df_analysis["netPnLUSD"] = df_analysis["pnlUSD"] - df_analysis["transactionCostUSD"].fillna(0)

print(f"\nTransaction costs calculated for {df_analysis['transactionCostUSD'].notna().sum()} transactions")
print(f"Average transaction cost: ${df_analysis['transactionCostUSD'].mean():.2f}")
print(f"Median transaction cost: ${df_analysis['transactionCostUSD'].median():.2f}")


Loaded 951 liquidation records
Columns: ['transactionHash', 'blockNumber', 'timestamp', 'datetime', 'liquidator', 'liquidatedUser', 'debtAsset', 'collateralAsset', 'debtToCover', 'liquidatedCollateralAmount', 'debtDecimals', 'collateralDecimals', 'debtUSD', 'collateralUSD', 'pnlUSD']

Fetching transaction costs...
Processed 10/951 transactions...
Processed 20/951 transactions...
Processed 30/951 transactions...
Processed 40/951 transactions...
Processed 50/951 transactions...
Processed 60/951 transactions...
Processed 70/951 transactions...
Processed 80/951 transactions...
Processed 90/951 transactions...
Processed 100/951 transactions...
Processed 110/951 transactions...
Processed 120/951 transactions...
Processed 130/951 transactions...
Processed 140/951 transactions...
Processed 150/951 transactions...
Processed 160/951 transactions...
Processed 170/951 transactions...
Processed 180/951 transactions...
Processed 190/951 transactions...
Processed 200/951 transactions...
Processed 210

In [64]:
## Summary Statistics for PnL

# Summary statistics for gross PnL
print("=" * 80)
print("GROSS PNL (Before Transaction Costs) SUMMARY STATISTICS")
print("=" * 80)
gross_pnl_stats = df_analysis["pnlUSD"].describe()
print(gross_pnl_stats)
print(f"\nTotal Gross PnL: ${df_analysis['pnlUSD'].sum():,.2f}")
print(f"Number of profitable liquidations: {(df_analysis['pnlUSD'] > 0).sum()}")
print(f"Number of unprofitable liquidations: {(df_analysis['pnlUSD'] <= 0).sum()}")

# Summary statistics for net PnL
print("\n" + "=" * 80)
print("NET PNL (After Transaction Costs) SUMMARY STATISTICS")
print("=" * 80)
net_pnl_stats = df_analysis["netPnLUSD"].describe()
print(net_pnl_stats)
print(f"\nTotal Net PnL: ${df_analysis['netPnLUSD'].sum():,.2f}")
print(f"Number of profitable liquidations (net): {(df_analysis['netPnLUSD'] > 0).sum()}")
print(f"Number of unprofitable liquidations (net): {(df_analysis['netPnLUSD'] <= 0).sum()}")

# Impact of transaction costs
print("\n" + "=" * 80)
print("TRANSACTION COST IMPACT")
print("=" * 80)
total_tx_costs = df_analysis["transactionCostUSD"].sum()
print(f"Total transaction costs: ${total_tx_costs:,.2f}")
print(f"Average transaction cost per liquidation: ${df_analysis['transactionCostUSD'].mean():.2f}")
print(f"Transaction costs as % of gross PnL: {(total_tx_costs / df_analysis['pnlUSD'].sum() * 100):.2f}%")


GROSS PNL (Before Transaction Costs) SUMMARY STATISTICS
count       945.000000
mean       1118.652526
std       17945.266902
min          -0.719601
25%           0.134823
50%           0.248486
75%           0.395077
max      376257.363965
Name: pnlUSD, dtype: float64

Total Gross PnL: $1,057,126.64
Number of profitable liquidations: 908
Number of unprofitable liquidations: 37

NET PNL (After Transaction Costs) SUMMARY STATISTICS
count       945.000000
mean       1108.699288
std       17944.853256
min        -170.204753
25%          -0.003588
50%           0.133684
75%           0.281855
max      376252.427567
Name: netPnLUSD, dtype: float64

Total Net PnL: $1,047,720.83
Number of profitable liquidations (net): 705
Number of unprofitable liquidations (net): 240

TRANSACTION COST IMPACT
Total transaction costs: $9,480.49
Average transaction cost per liquidation: $9.97
Transaction costs as % of gross PnL: 0.90%


## PnL Summary Statistics Visualizations

Visualize PnL summary statistics, distributions, and top liquidators.


In [79]:
# Clean PnL Visualizations with Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Create subplots: 1 row, 3 columns
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Key Statistics', 'Top 5 Liquidators by Total Net PnL', 'Distribution of Log PnL'),
    specs=[[{"type": "table"}, {"type": "bar"}, {"type": "histogram"}]],
    horizontal_spacing=0.12,
    column_widths=[0.35, 0.35, 0.30]
)

# 1. Key Statistics Table
stats_data = {
    'Metric': [
        'Total Liquidations',
        'Total Gross PnL',
        'Total Net PnL',
        'Total Tx Costs',
        'Mean Gross PnL',
        'Mean Net PnL',
        'Median Gross PnL',
        'Median Net PnL',
        'Std Dev Gross PnL',
        'Std Dev Net PnL',
        'Min Gross PnL',
        'Max Gross PnL',
        'Profitable (Gross)',
        'Profitable (Net)'
    ],
    'Value': [
        f'{len(df_analysis):,}',
        f'${df_analysis["pnlUSD"].sum():,.2f}',
        f'${df_analysis["netPnLUSD"].sum():,.2f}',
        f'${df_analysis["transactionCostUSD"].sum():,.2f}',
        f'${df_analysis["pnlUSD"].mean():,.2f}',
        f'${df_analysis["netPnLUSD"].mean():,.2f}',
        f'${df_analysis["pnlUSD"].median():,.2f}',
        f'${df_analysis["netPnLUSD"].median():,.2f}',
        f'${df_analysis["pnlUSD"].std():,.2f}',
        f'${df_analysis["netPnLUSD"].std():,.2f}',
        f'${df_analysis["pnlUSD"].min():,.2f}',
        f'${df_analysis["pnlUSD"].max():,.2f}',
        f'{(df_analysis["pnlUSD"] > 0).sum():,}',
        f'{(df_analysis["netPnLUSD"] > 0).sum():,}'
    ]
}

fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Metric</b>', '<b>Value</b>'],
            fill_color='steelblue',
            align='left',
            font=dict(size=14, color='white'),
            height=40
        ),
        cells=dict(
            values=[stats_data['Metric'], stats_data['Value']],
            fill_color=[['white', 'lightgray'] * len(stats_data['Metric'])],
            align='left',
            font=dict(size=12),
            height=30
        )
    ),
    row=1, col=1
)

# 2. Top 5 Liquidators by Total Net PnL
top_5_liquidators = liquidator_stats.head(5)
liquidator_labels = [addr[:10] + "..." for addr in top_5_liquidators.index]

fig.add_trace(
    go.Bar(
        x=top_5_liquidators['totalNetPnL'],
        y=liquidator_labels,
        orientation='h',
        marker=dict(color='steelblue', line=dict(color='black', width=1)),
        text=[f'${val:,.0f}' for val in top_5_liquidators['totalNetPnL']],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>Total Net PnL: $%{x:,.2f}<extra></extra>'
    ),
    row=1, col=2
)

# 3. Distribution of Log PnL (positive values only)
positive_pnl = df_analysis[df_analysis['pnlUSD'] > 0]['pnlUSD']
log_pnl = np.log10(positive_pnl)

fig.add_trace(
    go.Histogram(
        x=log_pnl,
        nbinsx=50,
        marker=dict(color='green', line=dict(color='black', width=1)),
        hovertemplate='Log10(PnL): %{x:.2f}<br>Count: %{y}<extra></extra>',
        name='Log PnL Distribution'
    ),
    row=1, col=3
)

# Update layout
fig.update_layout(
    title=dict(
        text='PnL Analysis Summary - All 951 Liquidations',
        x=0.5,
        font=dict(size=20, color='black')
    ),
    height=600,
    width=1800,
    showlegend=False,
    font=dict(size=12)
)

# Update axes
fig.update_xaxes(title_text="Total Net PnL (USD)", row=1, col=2)
fig.update_yaxes(title_text="Liquidator", row=1, col=2)
fig.update_xaxes(title_text="Log10(Gross PnL)", row=1, col=3)
fig.update_yaxes(title_text="Frequency", row=1, col=3)

# Save and show
fig.write_html(DATA_DIR / "pnl_summary_plotly.html")
print("PnL summary visualization saved to data/pnl_summary_plotly.html")
fig.show()


PnL summary visualization saved to data/pnl_summary_plotly.html


In [69]:
## Liquidator Statistics

# Analyze liquidator performance
liquidator_stats = df_analysis.groupby("liquidator").agg({
    "transactionHash": "count",  # Number of liquidations
    "pnlUSD": ["sum", "mean", "median"],  # Gross PnL stats
    "netPnLUSD": ["sum", "mean", "median"],  # Net PnL stats
    "transactionCostUSD": "sum",  # Total transaction costs
}).round(2)

# Flatten column names
liquidator_stats.columns = [
    "totalLiquidations",
    "totalGrossPnL",
    "avgGrossPnL",
    "medianGrossPnL",
    "totalNetPnL",
    "avgNetPnL",
    "medianNetPnL",
    "totalTxCosts",
]

# Sort by total net PnL
liquidator_stats = liquidator_stats.sort_values("totalNetPnL", ascending=False)

print("=" * 100)
print("LIQUIDATOR STATISTICS")
print("=" * 100)
print(f"\nTotal unique liquidators: {len(liquidator_stats)}")
print(f"\nTop 10 Liquidators by Total Net PnL:")
print(liquidator_stats.head(10).to_string())

# Summary statistics on liquidators
print("\n" + "=" * 100)
print("LIQUIDATOR SUMMARY STATISTICS")
print("=" * 100)
print(f"\nLiquidations per liquidator:")
print(liquidator_stats["totalLiquidations"].describe())
print(f"\nTotal Net PnL per liquidator:")
print(liquidator_stats["totalNetPnL"].describe())
print(f"\nAverage Net PnL per liquidator:")
print(liquidator_stats["avgNetPnL"].describe())

# Liquidator concentration analysis
print("\n" + "=" * 100)
print("LIQUIDATOR CONCENTRATION ANALYSIS")
print("=" * 100)
total_liquidations = liquidator_stats["totalLiquidations"].sum()
top_10_liquidations = liquidator_stats.head(10)["totalLiquidations"].sum()
top_10_pnl = liquidator_stats.head(10)["totalNetPnL"].sum()
total_pnl = liquidator_stats["totalNetPnL"].sum()

print(f"Top 10 liquidators account for {top_10_liquidations} liquidations ({top_10_liquidations/total_liquidations*100:.2f}%)")
print(f"Top 10 liquidators account for ${top_10_pnl:,.2f} in net PnL ({top_10_pnl/total_pnl*100:.2f}%)")

# Save liquidator statistics
liquidator_stats.to_csv(DATA_DIR / "liquidator_statistics.csv")
print(f"\nLiquidator statistics saved to data/liquidator_statistics.csv")


LIQUIDATOR STATISTICS

Total unique liquidators: 43

Top 10 Liquidators by Total Net PnL:
                                            totalLiquidations  totalGrossPnL  avgGrossPnL  medianGrossPnL  totalNetPnL  avgNetPnL  medianNetPnL  totalTxCosts
liquidator                                                                                                                                                   
0xcb7066289252875360Ad69CE71514AEc41D50993                  9      789287.69     87698.63          520.55    789263.11   87695.90        519.14         24.58
0x193e08Fd4375444017CAFF5f24C0FBC3f273f846                 17      127755.11      7515.01           48.44    127743.53    7514.33         48.32         11.58
0xE08D97e151473A848C3d9CA3f323Cb720472D015                171       60662.43       358.95            0.22     60647.82     358.86          0.13         14.79
0x88eb6e4358E940EeA0151E16C77C336370708729                  6       46548.85      7758.14          941.67     46535.66  

## Flash Loan and Collateral Swap Analysis by Liquidator

Analyze flash loan usage and collateral swaps by liquidator from detailed manipulation analysis.


In [ ]:
# Load detailed manipulation analysis
df_detailed = pd.read_csv(DATA_DIR / "detailed_manipulation_analysis.csv")

print("=" * 80)
print("FLASH LOAN ANALYSIS BY LIQUIDATOR")
print("=" * 80)

# Count liquidations with flash loans by liquidator
flash_loan_by_liquidator = df_detailed[df_detailed['flashLoanCount'] > 0].groupby('liquidator').agg({
    'liquidationTxHash': 'count',
    'flashLoanCount': 'sum',
    'pnlUSD': ['sum', 'mean']
}).round(2)

flash_loan_by_liquidator.columns = ['liquidations_with_flash_loans', 'total_flash_loans', 'total_pnl', 'avg_pnl']
flash_loan_by_liquidator = flash_loan_by_liquidator.sort_values('liquidations_with_flash_loans', ascending=False)

print(f"\nTotal liquidations with flash loans: {len(df_detailed[df_detailed['flashLoanCount'] > 0])}")
print(f"Total unique liquidators using flash loans: {len(flash_loan_by_liquidator)}")
print(f"\nTop liquidators by number of liquidations with flash loans:")
print(flash_loan_by_liquidator.head(20).to_string())

# Save to CSV
flash_loan_by_liquidator.to_csv(DATA_DIR / "flash_loan_by_liquidator.csv")
print(f"\nFlash loan analysis saved to data/flash_loan_by_liquidator.csv")


FLASH LOAN ANALYSIS BY LIQUIDATOR

Total liquidations with flash loans: 15
Total unique liquidators using flash loans: 6

Top liquidators by number of liquidations with flash loans:
                                            liquidations_with_flash_loans  total_flash_loans  total_pnl    avg_pnl
liquidator                                                                                                        
0x1f2f10d1c40777ae1da742455c65828ff36df387                              5                  9   12206.97    2441.39
0xcb7066289252875360ad69ce71514aec41d50993                              4                 10  787610.85  196902.71
0x88eb6e4358e940eea0151e16c77c336370708729                              3                  7   44921.00   14973.67
0x193e08fd4375444017caff5f24c0fbc3f273f846                              1                  1  125729.36  125729.36
0x982464e3c899c70305ce889ec59184201e5d7a2e                              1                  1    8697.15    8697.15
0xa7bc4d21680

In [73]:
print("\n" + "=" * 80)
print("COLLATERAL SWAP ANALYSIS BY LIQUIDATOR")
print("=" * 80)

# Count liquidations with collateral swaps by liquidator
collateral_swap_by_liquidator = df_detailed[df_detailed['collateralTokenSwapCount'] > 0].groupby('liquidator').agg({
    'liquidationTxHash': 'count',
    'collateralTokenSwapCount': 'sum',
    'pnlUSD': ['sum', 'mean']
}).round(2)

collateral_swap_by_liquidator.columns = ['liquidations_with_collateral_swaps', 'total_collateral_swaps', 'total_pnl', 'avg_pnl']
collateral_swap_by_liquidator = collateral_swap_by_liquidator.sort_values('liquidations_with_collateral_swaps', ascending=False)

print(f"\nTotal liquidations with collateral swaps: {len(df_detailed[df_detailed['collateralTokenSwapCount'] > 0])}")
print(f"Total unique liquidators using collateral swaps: {len(collateral_swap_by_liquidator)}")
print(f"\nTop liquidators by number of liquidations with collateral swaps:")
print(collateral_swap_by_liquidator.head(20).to_string())

# Save to CSV
collateral_swap_by_liquidator.to_csv(DATA_DIR / "collateral_swap_by_liquidator.csv")
print(f"\nCollateral swap analysis saved to data/collateral_swap_by_liquidator.csv")



COLLATERAL SWAP ANALYSIS BY LIQUIDATOR

Total liquidations with collateral swaps: 16
Total unique liquidators using collateral swaps: 7

Top liquidators by number of liquidations with collateral swaps:
                                            liquidations_with_collateral_swaps  total_collateral_swaps  total_pnl    avg_pnl
liquidator                                                                                                                  
0x1f2f10d1c40777ae1da742455c65828ff36df387                                   5                     137   12206.97    2441.39
0xcb7066289252875360ad69ce71514aec41d50993                                   4                     100  787610.85  196902.71
0x88eb6e4358e940eea0151e16c77c336370708729                                   3                      35   44921.00   14973.67
0x193e08fd4375444017caff5f24c0fbc3f273f846                                   1                       1  125729.36  125729.36
0xa7bc4d21680ab2d088614951b20c9195e8d23470     

In [75]:
print("\n" + "=" * 80)
print("COMBINED ANALYSIS: Flash Loans AND Collateral Swaps")
print("=" * 80)

# Liquidations with both flash loans AND collateral swaps
both_by_liquidator = df_detailed[(df_detailed['flashLoanCount'] > 0) & (df_detailed['collateralTokenSwapCount'] > 0)].groupby('liquidator').agg({
    'liquidationTxHash': 'count',
    'flashLoanCount': 'sum',
    'collateralTokenSwapCount': 'sum',
    'pnlUSD': ['sum', 'mean']
}).round(2)

both_by_liquidator.columns = ['liquidations_with_both', 'total_flash_loans', 'total_collateral_swaps', 'total_pnl', 'avg_pnl']
both_by_liquidator = both_by_liquidator.sort_values('liquidations_with_both', ascending=False)

print(f"\nTotal liquidations with BOTH flash loans AND collateral swaps: {len(df_detailed[(df_detailed['flashLoanCount'] > 0) & (df_detailed['collateralTokenSwapCount'] > 0)])}")
print(f"Total unique liquidators using both: {len(both_by_liquidator)}")
print(f"\nTop liquidators using both flash loans and collateral swaps:")
print(both_by_liquidator.head(20).to_string())




COMBINED ANALYSIS: Flash Loans AND Collateral Swaps

Total liquidations with BOTH flash loans AND collateral swaps: 14
Total unique liquidators using both: 5

Top liquidators using both flash loans and collateral swaps:
                                            liquidations_with_both  total_flash_loans  total_collateral_swaps  total_pnl    avg_pnl
liquidator                                                                                                                         
0x1f2f10d1c40777ae1da742455c65828ff36df387                       5                  9                     137   12206.97    2441.39
0xcb7066289252875360ad69ce71514aec41d50993                       4                 10                     100  787610.85  196902.71
0x88eb6e4358e940eea0151e16c77c336370708729                       3                  7                      35   44921.00   14973.67
0x193e08fd4375444017caff5f24c0fbc3f273f846                       1                  1                       1  125729.3